<a href="https://colab.research.google.com/github/bkristensen/Colabs/blob/main/Teo/Hent_synonymer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opsætning

## Installere moduler

In [7]:
!pip install gspread oauth2client pyppeteer nest_asyncio


## Importér moduler

In [8]:
import gspread
import sqlite3

import asyncio
import nest_asyncio

import pyppeteer

import google.auth

import urllib

nest_asyncio.apply()


## Funktioner

In [9]:
async def test_browser():
  browser = await pyppeteer.launch(headless=True, args=["--no-sandbox"])
  page = await browser.newPage()
  await page.goto("https://example.com")
  content = await page.content()
  await browser.close()
  print("Browser virker!")

async def initGoogleAccess():
  # Godkend adgang til Google sheets
  google.colab.auth.authenticate_user()
  google.colab.drive.mount(conf["drive_dir"])

async def hent_synonymer(ordet: str) -> list:
  synonymer: list = []
  browser: pyppeteer.browser.Browser = await pyppeteer.launch(headless=True, args=["--no-sandbox"])
  page: pyppeteer.browser.Page = await browser.newPage()
  url = f"https://synonymet.dk/ord/{urllib.parse.quote(ordet)}"
  await page.goto(url)
  try:
    await page.waitForSelector(".wordcloud-span", timeout=3000)
    elementer: list = await page.querySelectorAll(".wordcloud-span")
    for el in elementer:
      tekst = (await page.evaluate('(el) => el.textContent', el)).strip()
      if tekst:
        synonymer.append(tekst.capitalize())
  except Exception as e:
    elementer = []
  await browser.close()
  return synonymer

async def initDatabase():
  # Opret tabeller
  conn = sqlite3.connect(conf['database_dir'])
  cursor = conn.cursor()
  for script in dbscripts:
    cursor.executescript(script)
  conn.commit()
  cursor.close()
  conn.close()

def loadSheetData():
  # Åbn arket og vælg det første ark
  creds, _ = google.auth.default()
  gc = gspread.authorize(creds)
  sheet = gc.open("Synonymer").get_worksheet(2)
  # Hent alle ord fra kolonne A, start fra række 2
  ordliste_ids = sheet.col_values(1)[1:]
  ordliste_tags = sheet.col_values(2)[1:]
  return ordliste_ids, ordliste_tags

def gem_ord_med_synonymer(conn: sqlite3.Connection, ordet: str, tagid: int | None):
  # Indsæt ord (hvis det ikke allerede findes)
  new_tag: bool = False
  id: int | None = None
  fdata: list | None = None

  cursor: sqlite3.Cursor = conn.cursor()
  cursor.execute("SELECT Id FROM Tags WHERE Tag = ?", (ordet,))
  fdata = cursor.fetchone()

  if not fdata:
    new_tag = True
    print(f" - Indsætter ny tag: {ordet}")
    cursor.execute("INSERT OR IGNORE INTO Tags (Tag) VALUES (?)", (ordet,))
    cursor.execute("SELECT Id FROM Tags WHERE Tag = ?", (ordet,))
    id = cursor.fetchone()[0]
  else:
    id = fdata[0]

  if tagid:
    cursor.execute("INSERT OR IGNORE INTO TagIds (TId, TagId) VALUES (?, ?)", (id, tagid))

  if new_tag:
    print(f"      Henter synonymer for: {ordet}")
    syns: list = asyncio.get_event_loop().run_until_complete(hent_synonymer(ordet))
    if syns:
      print(f"         Antal synonymer for {ordet}: {len(syns)}")
      for synonym in syns:
        cursor.execute("INSERT OR IGNORE INTO Synonyms (Synonym) VALUES (?)", (synonym,))
        cursor.execute("SELECT Id FROM Synonyms WHERE Synonym = ?", (synonym,))
        synonym_id = cursor.fetchone()[0]

        # Link kun hvis ikke allerede linket
        cursor.execute("INSERT OR IGNORE INTO TagSynonyms (TId, SId) VALUES (?, ?)", (id, synonym_id))
    else:
      print(f"        Kunne ikke finde synonymer for {ordet}")
  conn.commit()
  cursor.close()



## Variabler

In [10]:
conf = {
  "drive_dir":      "/content/drive",
  "data_dir":       "/content/drive/MyDrive/Data",
  "database_dir":   "/content/drive/MyDrive/Data/TagSynonyms.db"
}
resultater = []

dbscripts = [
"""
CREATE TABLE IF NOT EXISTS Tags (
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tag STRING(100) UNIQUE NOT NULL
);
""",
"""
CREATE TABLE IF NOT EXISTS Synonyms (
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    Synonym STRING(100) UNIQUE NOT NULL
);
""",
"""
CREATE TABLE IF NOT EXISTS TagSynonyms (
    TId INTEGER NOT NULL,
    SId INTEGER NOT NULL,
    PRIMARY KEY (TId, SId),
    FOREIGN KEY (TId) REFERENCES Tags(Id),
    FOREIGN KEY (SId) REFERENCES Synonyms(Id)
);
""",
"""
CREATE TABLE IF NOT EXISTS TagIds (
    TId INTEGER NOT NULL,
    TagId INTEGER NOT NULL,
    PRIMARY KEY (TId, TagId),
    FOREIGN KEY (TId) REFERENCES Tags(Id)
);
""",
"""
DROP VIEW IF EXISTS vTags;
CREATE VIEW vTags AS
SELECT
	t.Id AS Id,
	t.Tag AS TagName,
	ti.TagId AS TagId
FROM
	Tags t
	LEFT OUTER JOIN TagIds ti
		ON ti.TId = t.Id
ORDER BY
	t.Tag,
	ti.TagId;
""",
"""
DROP VIEW IF EXISTS vSynonymTags;
CREATE VIEW vSynonymTags AS
SELECT
	COUNT(sc.Id) AS TotalTags,
	sc.Synonym,
	tc.TagId,
	tc.Tags
FROM
	vSynCount sc
	JOIN TagSynonyms ts
		ON ts.SId = sc.Id
	JOIN TagIds ti
		ON ti.TId = ts.TId
	JOIN vTagConcat tc
		ON tc.TagId = ti.TagId
GROUP BY
	sc.Id,
	tc.TagId
ORDER BY
	sc.Synonym,
	tc.Tags,
	TotalTags DESC;
""",
"""
DROP VIEW IF EXISTS vTagSynonyms;
CREATE VIEW vTagSynonyms AS
SELECT
	t.Tag AS TagName,
	s.Synonym AS Synonym,
	t.Id AS TId,
	ts.SId AS SId
FROM
	Tags t
	LEFT OUTER JOIN TagSynonyms ts
		ON ts.TId = t.Id
	LEFT OUTER JOIN Synonyms s
		ON s.Id = ts.SId
ORDER BY
	t.Tag,
	s.Synonym;
""",
"""
DROP VIEW IF EXISTS vTagSynonymIds;
CREATE VIEW vTagSynonymIds AS
SELECT
	ti.TagId AS TagId,
	t.Tag AS TagName,
	s.Synonym AS Synonym,
	t.Id AS TId,
	s.Id AS SId
FROM
	Tags t
	LEFT OUTER JOIN TagIds ti
		ON ti.TId = t.Id
	LEFT OUTER JOIN TagSynonyms ts
		ON ts.TId = t.Id
	LEFT OUTER JOIN Synonyms s
		ON s.Id = ts.SId
ORDER BY
	t.Tag,
	s.Synonym;
""",
"""
DROP VIEW IF EXISTS vTagConcat;
CREATE VIEW vTagConcat AS
SELECT
    ti.TagId,
    GROUP_CONCAT(t.Tag, '; ') AS Tags
FROM TagIds ti
JOIN Tags t ON ti.TId = t.Id
GROUP BY ti.TagId
ORDER BY ti.TagId, t.Tag;
""",
"""
DROP VIEW IF EXISTS vSynCount;
CREATE VIEW vSynCount AS
SELECT
	s.Id AS Id,
	s.Synonym AS Synonym,
	COUNT(*) AS TotalTags
FROM
	Synonyms s
	JOIN TagSynonyms ts
		ON ts.SId = s.Id
	JOIN TagIds ti
		ON ti.TId = ts.TId
GROUP BY s.Id
ORDER BY
	TotalTags DESC,
	s.Synonym;
""",
"""
DROP VIEW IF EXISTS vSynonymTagCount;
CREATE VIEW vSynonymTagCount AS
SELECT
	sc.Id,
	sc.Synonym,
	sc.TotalTags,
	tc.TagId,
	tc.Tags
FROM
	vSynCount sc
	JOIN TagSynonyms ts
		ON ts.SId = sc.Id
	JOIN TagIds ti
		ON ti.TId = ts.TId
	JOIN vTagConcat tc
		ON tc.TagId = ti.TagId
ORDER BY
	sc.TotalTags DESC,
	sc.Synonym,
	tc.Tags;
""",
"""
DROP VIEW IF EXISTS vSynonymTagStat;
CREATE VIEW vSynonymTagStat AS
SELECT * FROM vSynonymTags vst
WHERE vst.TotalTags > 1
ORDER BY
	vst.TotalTags DESC,
	vst.Synonym,
	vst.Tags;
"""]

## Init

In [11]:
await test_browser()

Browser virker!


In [12]:
await initGoogleAccess()

await initDatabase()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Gem ord og synonym

In [13]:
print(" - Henter ord fra Google Sheets")
ordliste_ids, ordliste_tags = loadSheetData()

conn = sqlite3.connect(conf['database_dir'])

print(f"""      Antal ord: {len(ordliste_tags)}
      """)

for i, ordet in enumerate(ordliste_tags):
  try:
    gem_ord_med_synonymer(conn, ordet.capitalize(), ordliste_ids[i])
  except Exception as e:
    print(f"         FEJL under tjek af {ordet} - Fejl: {e}")
    continue

print("""
    F Æ R D I G ! ! !""")
conn.close()

 - Henter ord fra Google Sheets
      Antal ord: 514
      
 - Indsætter ny tag: Blod
      Henter synonymer for: Blod
         Antal synonymer for Blod: 3
 - Indsætter ny tag: Lægebehandling
      Henter synonymer for: Lægebehandling
         Antal synonymer for Lægebehandling: 5
 - Indsætter ny tag: Dæmoner
      Henter synonymer for: Dæmoner
         Antal synonymer for Dæmoner: 11
 - Indsætter ny tag: Satan
      Henter synonymer for: Satan
         Antal synonymer for Satan: 17
 - Indsætter ny tag: Gamle
      Henter synonymer for: Gamle
         Antal synonymer for Gamle: 20
 - Indsætter ny tag: Gamle verden
      Henter synonymer for: Gamle verden
        FEJL under læsning af 'https://synonymet.dk/ord/Gamle%20verden': 
      Waiting for selector ".wordcloud-span" failed: timeout 3000ms exceeds.
        Kunne ikke finde synonymer for Gamle verden
 - Indsætter ny tag: Verden
      Henter synonymer for: Verden
         Antal synonymer for Verden: 39
 - Indsætter ny tag: Afgud
    